In [2]:
### General Imports 
import io
import string
import time
import os
import random
import pandas as pd
import numpy as np
import pickle


### IPY Widgets
from ipywidgets import HBox
from ipysheet import sheet, cell, column
# import bqplot.pyplot as plt
# from traitlets import link

### Q Grid
import qgrid

### IPY Sheet
from ipysheet import from_dataframe, to_dataframe
import ipysheet

### MITO (Free Version)
import mitosheet
from mitosheet import *

### STREAM LIT (NOT WORKING RN)
import streamlit as st
from st_aggrid import GridOptionsBuilder, AgGrid, GridUpdateMode, DataReturnMode

### IPY AG GRID
import ipyaggrid
from ipyaggrid import *

### This is what the dataframe looks like.

- `Company` and `Country` are the inputs.  
  
- `Imputation_Results_1`, `Imputation_Results_2`, `Imputation_Results_3` are the possible repairs (we set k=3)for the missing column which is `Industry Sector Missing`.  
  
- In this case, the repair suggestions are in order of confidence. `Imputation_Results_1` the repair with >= confidence as the result in `Imputation_Results_2` and so on.  
  
- In `Industry Sector Missing` some values are missing ( = NaN ) and some are not missing ( != NaN and are already shown). Ignore the predictions for the rows that don't have the missing value. 

In [7]:
new_ouput_dataframe_interaction_df = pd.read_csv("Imp_library_code/visualization_interactive_test_df.csv")
possible_cats = ['Vehicles & Components',
 'Consumer Goods',
 'Food, Beverage & Tobacco',
 'Utilities',
 'Retailers',
 'Industrial Goods',
 'Textiles, Apparel & Luxury Goods',
 'Technology Equipment',
 'Materials',
 'Hotels & Restaurants',
 'Telecommunications',
 'Transportation & Logistics',
 'Information Technology & Services',
 'Media & Publishing',
 'Energy',
 'Aerospace & Defense',
 'Financials',
 'Healthcare']

possible_cats.sort()
new_ouput_dataframe_interaction_df['Final Value 2'] = pd.Categorical([""]*new_ouput_dataframe_interaction_df.shape[0], categories=possible_cats)
new_ouput_dataframe_interaction_df = new_ouput_dataframe_interaction_df[['Company', 'Country', 'Imputation_Results_1',
       'Imputation_Results_2', 'Imputation_Results_3',
       'Industry Sector Missing', 'Final Value 2']]

new_ouput_dataframe_interaction_df.columns = ['Company', 'Country', 'Imputation_Results_1',
       'Imputation_Results_2', 'Imputation_Results_3',
       'Industry Sector Missing', 'Final Chosen Value']

new_ouput_dataframe_interaction_df



,Company,Country,Imputation_Results_1,Imputation_Results_2,Imputation_Results_3,Industry Sector Missing,Final Chosen Value
0,Santander Brasil,Brazil,Financials,Financials,Financials,Financials,NaN
1,Wipro,India,Information Technology & Services,Information Technology & Services,Information Technology & Services,Information Technology & Services,NaN
2,Bradesco,Brazil,Financials,"Textiles, Apparel & Luxury Goods",Retailers,NaN,NaN
3,IBM,United States,Retailers,Industrial Goods,Financials,NaN,NaN
4,National Australia Bank,Australia,Financials,"Textiles, Apparel & Luxury Goods",Healthcare,NaN,NaN
...,...,...,...,...,...,...,...
15,Santander,Spain,Financials,Financials,Financials,Financials,NaN
16,Fujitsu,Japan,Information Technology & Services,Technology Equipment,Telecommunications,NaN,NaN
17,NKSJ Holdings,Japan,Retailers,Industrial Goods,Financials,NaN,NaN
18,Infosys,India,Retailers,Information Technology & Services,Industrial Goods,NaN,NaN


### Option 1: Q-GRID

- Simple enough user interface. 
- Can filter columns (useful because can filter `Industry Sector Missing` to only show the rows with NaN and then impute those.
- `Final Chosen Value` is a dropdown for each row (double click the cell). Dropdown values will be all the possible Industry Sector Values (not just the 3 predictions for that row), there is no way to change this. Can be tricky when number of possible categories is very high. 
- Saving the edits is very easy, (1 line of code which can be made as a function)

In [8]:
qgrid_widget = qgrid.QgridWidget(df=new_ouput_dataframe_interaction_df, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Option 2: MITO SHEET

- Slightly more complicated user interface but has more functionalities (although most of those features are useless in our use-case).
- Can filter and give summary stats for each col (again, not really sure if the stats are useful in this case).
- Can add, delete, rows and cols + can sort rows. 
- No dropdown imputation option. Will need to select each NaN cell in `Industry Sector Missing` and copy-paste/type in the chosen repair for that row.
- Has a Pro version which is paid (but the pro version does not provide the dropdown functionality or any other useful tool for us other than being able to color certain columns)
- Saving the changes requires more lines of code

In [12]:
mitosheet.sheet(new_ouput_dataframe_interaction_df, analysis_to_replay="id-fxxrnhgeae")

MitoWidget(analysis_data_json='{"analysisName": "id-fxxrnhgeae", "analysisToReplay": null, "code": [], "stepSu…

### Option 3: IPY AG GRID

`WHEN USING THE TABLE BELOW< BE SURE TO TOGGLE THE EDIT BUTTON TO ON. Once changes are made, click, Export Grid, and then run the next cell of code to see the updated dataframe`

- Clean user interface with relevant functionalities provided.
- Allows you to move through table values using keyboard keys (very useful for people who are used to typing and using keyboard shortkeys which most people (myself included) are. It makes the process very easy and fast.
- We can give some customization like pagentation number (or whether to have it or not). 
- Gives additional features like directly exporting to csv file and downloading.
- Filtering for NaNs in missing value column is a bit hard. But this can be fixed by simple clicking the column name and sorting them (this would put all the missing value rows at the top). 
- No column/row coloring options (just a few different themes). 

In [58]:
column_defs = [{'field': c} for c in new_ouput_dataframe_interaction_df.columns[:]]

grid_options = {
    'columnDefs' : column_defs,
     'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
#     "navigateToNextCell" : True,
    "pagination" : True,
    'paginationPageSize' : 10,
    "animateRows" : True,
    "enableCellEditingOnBackspace" : True,
    "rowDrag":True,
    'rowDragManaged' : True
#     "groupRowsSticky" : True,
#     "rowGroupPanelShow" : True,
#     "suppressRowDrag" : True
}

g = Grid(grid_data=new_ouput_dataframe_interaction_df,
         grid_options=grid_options,
         quick_filter=True,
         show_toggle_edit=True,
         export_mode="buttons",
         export_csv=True,
         export_excel=False,
         theme='ag-theme-fresh', # 'ag-theme-balham', 'ag-theme-balham-dark', 'ag-theme-material', 'ag-theme-fresh', 'ag-theme-dark', 'ag-theme-blue', 'ag-theme-bootstrap', 'ag-theme-excel'
         show_toggle_delete=True,
         columns_fit='auto',
         index=False, 
         keep_multiindex=False)
g

Grid(columns_fit='auto', compress_data=True, export_mode='buttons', height='350px', menu={'buttons': [{'name':…

In [43]:
g.grid_data_out.get('grid').head(20)

,Company,Country,Imputation_Results_1,Imputation_Results_2,Imputation_Results_3,Industry Sector Missing,Final Chosen Value
Index,,,,,,,
0,Santander Brasil,Brazil,Financials,Financials,Financials,Financials,None
1,Wipro,India,Information Technology & Services,Information Technology & Services,Information Technology & Services,Information Technology & Services,Information Technology & Services
2,Bradesco,Brazil,Financials,"Textiles, Apparel & Luxury Goods",Retailers,None,Retailers
3,IBM,United States,Retailers,Industrial Goods,Financials,None,Industrial Goods
4,National Australia Bank,Australia,Financials,"Textiles, Apparel & Luxury Goods",Healthcare,None,Financials
...,...,...,...,...,...,...,...
15,Santander,Spain,Financials,Financials,Financials,Financials,None
16,Fujitsu,Japan,Information Technology & Services,Technology Equipment,Telecommunications,None,None
17,NKSJ Holdings,Japan,Retailers,Industrial Goods,Financials,None,None


### Option 4: Make a seperate JavaScript Redirect Page with Dynamic JS tables

- Almost limitless Flexibility in terms of what can be done. Row specific dropdowns, table movements, coloring, etc are all possible. 

**Cons:**   
- Will take much more time to implement 
- Requires a server to handle requests (with a database to save states) etc. Basically, architecture becomes much more complex and that also will take some time to make (doable, but time consuming).
- Won't run locally on client/user's end, will need a server to run.
  
In my opinion, this is an option we can look towards in the future if we decide to scale or improve this system. 
